In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta

# Source files directory
input_directory = r'C:\Users\Meu Computador\OneDrive - Thirty Capital\01_Live Clients\09_Century Housing\Boston Post\Rent Roll - File Conversion\10.21'

# Destination directory where the new files will be saved
output_directory = r'C:\Users\Meu Computador\OneDrive - Thirty Capital\01_Live Clients\09_Century Housing\Boston Post\Rent Roll - File Conversion\10.21'

# Function to format dates, handling empty cells
def format_date(date_value):
    if pd.isna(date_value) or date_value == '':
        return ''  # Keeps the cell empty if the original value is blank
    try:
        date = pd.to_datetime(date_value, errors='coerce')
        if pd.isna(date):
            return ''  # Leaves blank if not a valid date
        return date.strftime('%m/%d/%Y')
    except:
        return ''  # Leaves blank if an error occurs

# Function to format numbers with two decimal places
def format_number(value):
    try:
        return f"{safe_float(value):.2f}"
    except:
        return ''  # Keeps the cell empty if there's an error

# Function to safely convert values to float with error handling
def safe_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return float('nan')  # Returns NaN to ensure empty cells remain empty

# Function to identify property name based on file name
def identify_property_name(file_name):
    file_name = file_name.lower().replace(' ', '')
    if 'academyhall' in file_name:
        return 'Academy Hall'
    elif 'casarita' in file_name:
        return 'Casa Rita'
    elif 'costamesa' in file_name:
        return 'Costa Mesa Village'
    elif 'florencemorehouse' in file_name:
        return 'Florence Morehouse'
    else:
        return 'Unknown Property'

# Function to generate file names with a fixed timestamp for each day of the week
def generate_file_names(property_name, start_date):
    # Converts the start date from a string to a datetime object
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    
    # Generates files for 7 days, from Monday to Sunday, with a fixed timestamp
    file_names = []
    for i in range(7):
        day_date = start_date + timedelta(days=i)
        timestamp = day_date.strftime('%Y-%m-%d') + '-T08_01_47.000'
        file_name = f"{timestamp}_rent_roll_{property_name}.csv"
        file_names.append(file_name)
    
    return file_names

# Function to clean the Lease To by removing zeros and unnecessary hours
def clean_lease_to(lease_to_value):
    if lease_to_value == '0' or lease_to_value == '':
        return ''  # Removes zeros and keeps the cell empty
    try:
        # Attempts to convert to date to remove the hour
        date = pd.to_datetime(lease_to_value, errors='coerce')
        if pd.isna(date):
            return lease_to_value  # If not a valid date, returns the original value
        return date.strftime('%m/%d/%Y')  # Returns only the date in MM/DD/YYYY format
    except:
        return lease_to_value  # If unable to convert, returns the original value

# Process files in the source folder
def process_files(input_directory, start_date):
    for file in os.listdir(input_directory):
        if file.endswith('.xls'):
            try:
                file_path = os.path.join(input_directory, file)
                print(f"Processing file: {file_path}")

                # Reading the Excel file without automatic date conversion
                df = pd.read_excel(file_path, header=None, dtype=str)  # Reading all as string
                print("File read successfully")

                # Identify property name based on the file name
                property_name = identify_property_name(file)
                print(f"Property name adjusted to: {property_name}")

                # Filter rows from column A
                filtered_df = df[df[0].astype(str).str.match(r'^[A-Z]-\d+')]  # First rule
                if filtered_df.empty:  # If not found, apply the second rule
                    filtered_df = df[df[0].astype(str).str.match(r'^\d{1,3}$')]  # Changed to allow up to 3 digits
                
                # Fill NaN values with 0 in the filtered DataFrame
                filtered_df = filtered_df.fillna(0)

                # Structure of the final DataFrame with header in the first row
                df_final = pd.DataFrame(columns=[
                    'Unit', 'BD/BA', 'Tenant', 'Status', 'Sq. Ft.', 'Market Rent', 'Rent', 'Deposits', 
                    'Lease From', 'Lease To', 'Move-in', 'Move-out', 'Past Due', 
                    'NSF Count', 'Late Count', 'Next Rent Increase Date', 'Next Rent Increase Amount'
                ])

                # Insert header in the first row and leave the second row blank
                df_final.loc[0] = df_final.columns
                df_final.loc[1] = [''] * len(df_final.columns)  # Blank row 2
                df_final.loc[2, 'Unit'] = property_name

                # Iterate over rows in the filtered DataFrame and map the data
                for index, row in filtered_df.iterrows():
                    unit = str(row[0])  # Unit: Apt Nbr (column A)
                    tenant = str(row[1])  # Tenant Name (column B)
                    bd_ba = str(row[8])  # BD/BA: Unit Class (column I)
                    sq_ft = str(row[9])  # Sq. Ft. (column J)
                    
                    # Corrected logic to properly identify status "R" as "Current"
                    status = 'Current' if str(row[10]).strip().upper() == 'R' else 'Vacant-Unrented'  # Status (column K)
                    
                    market_rent = format_number(row[11])  # Market Rent (column L)
                    rent = format_number(safe_float(row[13]) + safe_float(row[14]))  # Sum of columns N and O for Rent
                    deposits = format_number(safe_float(row[16]) + safe_float(row[17]))  # Sum of columns Q and R
                    lease_from = format_date(row[5])  # Lease From and Move-in (column F)
                    lease_to = clean_lease_to(str(row[7]))  # Lease To, cleaned to remove zeros and hours
                    past_due = format_number(row[18])  # Past Due (column S)

                    # Adding data to the final sheet starting from row 4
                    df_final.loc[len(df_final)] = [
                        unit, bd_ba, tenant, status, sq_ft, market_rent, rent, deposits, 
                        lease_from, lease_to, lease_from, '', past_due, '0', '0', '', ''
                    ]

                # Inserting the totals row at the end
                df_final.loc[len(df_final)] = [
                    'Totals', '', '', '', format_number(df_final['Sq. Ft.'].apply(safe_float).sum()), 
                    format_number(df_final['Market Rent'].apply(safe_float).sum()), 
                    format_number(df_final['Rent'].apply(safe_float).sum()), 
                    format_number(df_final['Deposits'].apply(safe_float).sum()), 
                    '', '', '', '', 
                    format_number(df_final['Past Due'].apply(safe_float).sum()), '0', '0', '', ''
                ]
                print("Data processed and totals row added")

                # Generate file names for each day of the week
                file_names = generate_file_names(property_name, start_date)
                
                # Save the final file for each day of the week in the destination folder
                for output_file_name in file_names:
                    output_path = os.path.join(output_directory, output_file_name)
                    df_final.to_csv(output_path, index=False, header=False)
                    print(f"File saved as: {output_file_name}")

            except Exception as e:
                print(f"An error occurred while processing the file {file}: {e}")

# Usage example
start_date = '2024-10-14'  # Start date provided by the user
process_files(input_directory, start_date)


Processing file: C:\Users\Meu Computador\OneDrive - Thirty Capital\01_Live Clients\09_Century Housing\Boston Post\Rent Roll - File Conversion\10.21\10-21-2024 rentroll academy hall.xls
File read successfully
Property name adjusted to: Academy Hall
Data processed and totals row added


C:\Users\Meu Computador\AppData\Local\Temp\ipykernel_21108\2150809522.py:101: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filtered_df = filtered_df.fillna(0)
C:\Users\Meu Computador\AppData\Local\Temp\ipykernel_21108\2150809522.py:101: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filtered_df = filtered_df.fillna(0)


File saved as: 2024-10-14-T08_01_47.000_rent_roll_Academy Hall.csv
File saved as: 2024-10-15-T08_01_47.000_rent_roll_Academy Hall.csv
File saved as: 2024-10-16-T08_01_47.000_rent_roll_Academy Hall.csv
File saved as: 2024-10-17-T08_01_47.000_rent_roll_Academy Hall.csv
File saved as: 2024-10-18-T08_01_47.000_rent_roll_Academy Hall.csv
File saved as: 2024-10-19-T08_01_47.000_rent_roll_Academy Hall.csv
File saved as: 2024-10-20-T08_01_47.000_rent_roll_Academy Hall.csv
Processing file: C:\Users\Meu Computador\OneDrive - Thirty Capital\01_Live Clients\09_Century Housing\Boston Post\Rent Roll - File Conversion\10.21\10-21-2024 rentroll costa mesa.xls
File read successfully
Property name adjusted to: Costa Mesa Village
Data processed and totals row added
File saved as: 2024-10-14-T08_01_47.000_rent_roll_Costa Mesa Village.csv
File saved as: 2024-10-15-T08_01_47.000_rent_roll_Costa Mesa Village.csv
File saved as: 2024-10-16-T08_01_47.000_rent_roll_Costa Mesa Village.csv
File saved as: 2024-10-1

C:\Users\Meu Computador\AppData\Local\Temp\ipykernel_21108\2150809522.py:101: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filtered_df = filtered_df.fillna(0)
C:\Users\Meu Computador\AppData\Local\Temp\ipykernel_21108\2150809522.py:101: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  filtered_df = filtered_df.fillna(0)
